<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Notebook Help

| What                  |Keyboard Shortcut |
|:-|:-|
| Run a cell            |Shift+Enter       |
| Restart python kernel |Esc+0+0           |
| Interrupt kernel      |Esc+i+i           |

In [ ]:
import copy
import imp
import pandas as pd
import yaml

In [ ]:
import araalictl
import api
import meta_policy
import meta_policy_gcp
import meta_policy_metadata_svc
import meta_policy_araali
import meta_policy_k8s
import meta_policy_ubuntu
import meta_policy_docker
import meta_policy_aws
import meta_policy_lacework
import meta_policy_logdna
import meta_policy_signalfx

imp.reload(araalictl)
imp.reload(api)
imp.reload(meta_policy)
imp.reload(meta_policy_gcp)
imp.reload(meta_policy_metadata_svc)
imp.reload(meta_policy_araali)
imp.reload(meta_policy_k8s)
imp.reload(meta_policy_ubuntu)
imp.reload(meta_policy_docker)
imp.reload(meta_policy_aws)
imp.reload(meta_policy_lacework)
imp.reload(meta_policy_logdna)
imp.reload(meta_policy_signalfx)

In [ ]:
tenant=""

In [ ]:
pd.DataFrame(araalictl.get_agents(tenant))

In [ ]:
apps = list(araalictl.get_apps(tenant))

In [ ]:
pd.DataFrame(apps)["zone"].value_counts()

In [ ]:
autok8s = [a for a in apps if "araali-operator" in a["app"]["name"]][0]

In [ ]:
autok8s

In [ ]:
def test_svc(dst, port):
    return araalictl.ping(autok8s["zone"], autok8s["app"]["name"], dst, port,
               autok8s["agent"], tenant)

In [ ]:
test_svc("169.254.169.254", 80)

In [ ]:
apps[0]

In [ ]:
def find_za_reps(apps):
    z_a = {}
    for a in apps:
        if a["app"]["name"] == "pause-container": continue
        z_a[(a["zone"], a["app"]["name"].split(".", 1)[0], a["agent"])] = a["app"]["name"]
    return z_a

def test_svc_reachability(dst, port):
    for ((zone, app, agent), full_app) in find_za_reps(araalictl.get_apps(tenant)).items():
        yield (zone, app, araalictl.ping(zone, full_app, dst, port, agent, tenant))

In [ ]:
pd.DataFrame(test_svc_reachability("169.254.169.254", 80))

In [ ]:
[a for a in apps if "cassandra" in a["app"]["name"]]

In [ ]:
araalictl.ping("bhanu", "ra-boutique.cartservice.server", "www.facebook.com", 80,
               "gtor.Fog.fog.bga.35e242d6340795b216659df3cc201bd400000622", "bga")

# Assessment

## Load it up

In [ ]:
%%time
tenant="sc"
tenant="letsbloomio"
tenant="hungerbox"
tenant="euler"
tenant="fiddler"
tenant="digitizethings"
tenant="informatica"
tenant="reltio"
tenant=""

imp.reload(araalictl)
imp.reload(api)
api.Runtime.zone2apps = None                                                    
run = api.Runtime(tenant) 
run = run.refresh()
runlinks = list(run.iterlinks())

In [ ]:
tenant

In [ ]:
api.Table(api.Runtime.get_zone_apps(hard=False))

## Cleanup Chatty Egress Links

In [ ]:
class DnsCleaner:
    policies = [
        api.AcceptLink(filters=[
                api.f.type("NAE"),
                api.f.endpoint("process", ["/usr/bin/dnf", "/bin/dnf",
                                           "packagekitd", "gnome-",
                                          ], who="client"),
                api.f.endpoint("dns_pattern", ":.*.:", who="server"),
            ], changes=[
            ]),       
    ]

In [ ]:
api.LinkTable(api.MetaPolicyRunner(DnsCleaner).run(runlinks).review(DnsCleaner))

## World exposed services

In [ ]:
api.Table(run.process_stats(all=True),
          lambda x: x["in degree"] > 0,
          lambda x: x["world exposed"] == True,
         )

In [ ]:
api.Table(run.process_stats(all=False),
          lambda x: x["in degree"] > 0,
          lambda x: x["world exposed"] == True,
         )

## Home exposed services

In [ ]:
api.Table(api.process_stats(run.iterlinks(), all=True),
          lambda x: x["in degree"] > 0,
          lambda x: x["home exposed"] == True,
          #lambda x: x["araali exposed"] == True,
         )

In [ ]:
api.Table(find_process("com.araalinetworks.LaunchKt"))

## Araali exposed services

In [ ]:
imp.reload(api)

In [ ]:
#api.Table(run.process_stats(all=True))
api.Table(api.process_stats(run.iterlinks(), all=True),
          lambda x: x["in degree"] > 0,
          lambda x: x["pcr"] > 0.7,
          #lambda x: x["out degree"] == 0,
          lambda x: x["araali exposed"] == True,
         )

In [ ]:
t = _

In [ ]:
%matplotlib inline

In [ ]:
s = pd.Series([a["pcr"] for a in t.links])

In [ ]:
s.plot.hist()

## Outside services

In [ ]:
api.Table(api.dns_stats(only_new=True, runlink=runlinks))

In [ ]:
pd.set_option('display.max_colwidth', 500)

In [ ]:
api.Table(run.dns_stats(only_new=True, runlink=runlinks),
          #lambda x: ".com" not in x["dns"] and ".net" not in x["dns"],
         )

In [ ]:
api.Table(run.server_stats(only_new=True, runlink=runlinks))

In [ ]:
imp.reload(api)

In [ ]:
api.Table(api.server_stats(only_new=True, runlink=runlinks))

In [ ]:
api.LinkTable(runlinks,
              api.f.endpoint("endpoint_group", "__WORLD__"), 
              api.f.endpoint("process", "/araali_backend.py", who="server"),              
              #api.f.endpoint("subnet", "169.254.169.254", who="server"),
              #api.f.neg(api.f.endpoint("dns_pattern", None, who="server")),
              #api.f.neg(api.f.endpoint("app", "gidus-01")),
              #api.f.neg(api.f.endpoint("app", "idev-01")),
              #api.f.neg(api.f.endpoint("app", "kube-system")),
              #api.f.neg(api.f.endpoint("app", "linux")),
              #api.f.endpoint("dns_pattern", "datastore.googleapis.com"),
              #api.f.neg(api.f.endpoint("dns_pattern", None, who="server")),
             )

In [ ]:
_.links[0]

In [ ]:
_.meta_policy(0)

# Vulnerabilities

In [ ]:
%%time
vulns = {}
cvulns = []
for za in api.Runtime.get_zone_apps(hard=False):
    z = za["name"]
    for a in za["Apps"]:
        for comp in araalictl.get_compute(zone=z, app=a, tenant=tenant):
            if comp["state"] != 1: continue
            maxv = 0
            for v in comp.get("vulnerabilities", []):
                maxv = max(v.get("severity", 0), maxv)
            cvulns.append({"zone": z, "app": a, 
                           "ip": comp["ip_address"], 
                           "os": comp["os_name"],
                           "container": comp["image"], 
                           "level": vlevel[maxv]})
            vulns[maxv] = vulns.get(maxv, 0) + 1

In [ ]:
vlevel = {0: "NONE", 1: "LOW", 2: "MEDIUM", 3: "HIGH", 4: "CRITICAL"}
pd.DataFrame([{"vulnerability level": vlevel[k], "asset count": v} for k,v in vulns.items()])

In [ ]:
pd.DataFrame(cvulns)[lambda df: df.level == "NONE"][["os"]].value_counts()

In [ ]:
pd.DataFrame(cvulns)[lambda df: df.level == "NONE"]

In [ ]:
link = "https://console-meta.aws.araalinetworks.com/policies?zone=prod&app=dmzvm&start-time=2021-02-22T04%3A17%3A31.955Z&perimeter-ingress-pins=__WORLD__&_other=reset"
from IPython.display import HTML
HTML("<a href=%s>world exposure</a>" % link)

In [ ]:
api.Table(find_process("vault-k8s"))

# Installing Python Packages into Notebook

In [ ]:
# This is needed only one time, to make sure all dependent packages are installed
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install oyaml
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib